In [7]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [9]:
# 규원님이 전처리 완료하신 최종 데이터 (+라벨까지 붙은)
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/data/dataset_210626_215600.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
len(df)

74779

In [10]:
df.head()

,gender,name,accords,review,tokenized,only_english,longer_than_2_A,stopwords_removed,lemmatizated,label
0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'sample', 'this', 'today', 'and', 'yea...","['got', 'sample', 'today', 'year', 'old', 'dau...","['get', 'sample', 'today', 'year', 'old', 'dau...",2.0
1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\r\nI have ...,"['first', 'impressions.', 'test', 'on', 'blott...","['first', 'impressions', 'test', 'on', 'blotte...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impression', 'test', 'blotter', 'he...",2.0
2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,"['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'best', 'friend...","['perfume', 'reminds', 'best', 'friend', 'actu...","['perfume', 'reminds', 'best', 'friend', 'actu...",2.0
3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,"['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'feet', 'falling', 'fa...","['imagine', 'trip', 'foot', 'fall', 'face', 'f...",2.0
4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\r\nLove the scent.....,"['gorgeous', 'gorgeous', 'blend', '..', 'love'...","['gorgeous', 'gorgeous', 'blend', '', 'love', ...","['gorgeous', 'gorgeous', 'blend', 'love', 'the...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...",2.0


본 데이터는 모두 string 형태라 전처리 필요

In [11]:
sent_text = df['lemmatizated']

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [14]:
for line in result[:3]:
  print(line)

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
['first', 'impression', 'test', 'blotter', 'heard', 'smell', 'like', 'bag', 'spray', 'like', 'gabriella', 'sabatini', 'perfume', 'everybody', 'past', 'woman', 'wear', 'smell', 'addictive', 'plasticky', 'beautiful', 'amaze', 'projection', 'longevity', 'woman', 'strong', 'personality', 'wear', 'may', 

### Word2Vec 훈련시키기

하이퍼 파라미터
* size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
* window = 컨텍스트 윈도우 크기
* min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
* workers = 학습을 위한 프로세스 수
* sg = 0은 CBOW, 1은 Skip-gram.

In [32]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec): 
  """Callback to print loss after each epoch."""
  def __init__(self):
    self.epoch = 0
    self.loss_to_be_subed = 0
  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    loss_now = loss - self.loss_to_be_subed
    self.loss_to_be_subed = loss
    print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
    self.epoch += 1


model = Word2Vec(sentences=result, size=200, window=10, min_count=50, workers=4, sg=1, compute_loss=True, iter=10, callbacks=[callback()])

Loss after epoch 0: 13671863.0
Loss after epoch 1: 11082395.0
Loss after epoch 2: 10164362.0
Loss after epoch 3: 6482432.0
Loss after epoch 4: 6469248.0
Loss after epoch 5: 6438312.0
Loss after epoch 6: 6558904.0
Loss after epoch 7: 6242988.0
Loss after epoch 8: 51184.0
Loss after epoch 9: 35448.0


### 수행 후 결과

In [33]:
model_result = model.wv.most_similar("fruity")
print(model_result)

[('citrusy', 0.6569720506668091), ('woody', 0.6541179418563843), ('citrus', 0.6353998184204102), ('sweet', 0.623024582862854), ('floral', 0.6227617859840393), ('fruit', 0.6179110407829285), ('juicy', 0.6131470203399658), ('pineapple', 0.5915230512619019), ('musky', 0.5905256271362305), ('spicy', 0.5672816038131714)]


In [34]:
model_result = model.wv.most_similar("aquatic")
print(model_result)

[('citrus', 0.6267548203468323), ('marine', 0.6002474427223206), ('fresh', 0.5947590470314026), ('blue', 0.5586378574371338), ('salty', 0.5568100810050964), ('watery', 0.5537587404251099), ('refresh', 0.5422773361206055), ('summery', 0.5418009161949158), ('citrusy', 0.5399379730224609), ('aqua', 0.5377556085586548)]


In [35]:
model_result = model.wv.most_similar("tobacco")
print(model_result)

[('vanille', 0.7132464647293091), ('pipe', 0.6359517574310303), ('honey', 0.5986115336418152), ('cinnamon', 0.5663580894470215), ('chergui', 0.553518533706665), ('spice', 0.5533519387245178), ('vanilla', 0.5413663983345032), ('cigar', 0.4884105324745178), ('boozy', 0.4554241895675659), ('hay', 0.4537080228328705)]


In [36]:
model_result = model.wv.most_similar("floral")
print(model_result)

[('florals', 0.7032735347747803), ('powdery', 0.6505076885223389), ('woody', 0.6504331231117249), ('citrus', 0.6359738111495972), ('soft', 0.6264920234680176), ('fruity', 0.6227617859840393), ('musky', 0.620571494102478), ('flowery', 0.6194251179695129), ('musk', 0.6027885675430298), ('jasmine', 0.5941770076751709)]


In [37]:
model_result = model.wv.most_similar("orange")
print(model_result)

[('blossom', 0.7775411605834961), ('mandarin', 0.649997353553772), ('peel', 0.6378258466720581), ('grapefruit', 0.5829253196716309), ('bergamot', 0.5644997358322144), ('jasmine', 0.5401254892349243), ('citrus', 0.5338088870048523), ('bitter', 0.5075477361679077), ('lemon', 0.4933609962463379), ('osmanthus', 0.488817423582077)]


### 모델 저장 및 로드

In [24]:
model.wv.save_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/model/w2v_200dim') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/model/w2v_200dim') # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("orange")
print(model_result)

[('blossom', 0.8549913167953491), ('mandarin', 0.7902723550796509), ('oranges', 0.7773942947387695), ('peel', 0.7708313465118408), ('grapefruit', 0.7638823390007019), ('zest', 0.7583459615707397), ('mandarine', 0.7549320459365845), ('osmanthus', 0.7393244504928589), ('creamsicle', 0.7385150790214539), ('gingery', 0.7377530336380005)]


In [25]:
print(loaded_model.wv.similarity("ocean", "summer"))

0.31355172


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [26]:
print(loaded_model.wv.similarity("forest", "woody"))

0.32360798


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [27]:
print(loaded_model.wv.similarity("jasmine", "floral"))

0.54451287


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [31]:
print(loaded_model.wv.similarity("woody", "forest"))

0.32360798


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


### 모델 저장 및 로드

In [38]:
model.wv.save_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/model/w2v_200dim_10window') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/model/w2v_200dim_10window') # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("orange")
print(model_result)

[('blossom', 0.8549913167953491), ('mandarin', 0.7902723550796509), ('oranges', 0.7773942947387695), ('peel', 0.7708313465118408), ('grapefruit', 0.7638823390007019), ('zest', 0.7583459615707397), ('mandarine', 0.7549320459365845), ('osmanthus', 0.7393244504928589), ('creamsicle', 0.7385150790214539), ('gingery', 0.7377530336380005)]


In [39]:
print(loaded_model.wv.similarity("ocean", "summer"))

0.28309456


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [40]:
print(loaded_model.wv.similarity("forest", "woody"))

0.3160655


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [41]:
print(loaded_model.wv.similarity("jasmine", "floral"))

0.594177


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [47]:
print(loaded_model.wv.similarity("male", "female"))

0.54936004


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
